In [ ]:
!pip install datasets transformers

In [ ]:
task = "pos" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "HooshvareLab/bert-fa-zwnj-base"
batch_size = 16

In [ ]:
from datasets import load_metric
import pandas as pd
import numpy as np

In [ ]:
raw_df = pd.read_csv("./drive/MyDrive/University/Thesis/Collection UNI.txt", sep="\n+", header=None)

In [ ]:
raw_tokens = pd.DataFrame(raw_df[0].str.rsplit(n = 1, expand=True)[[0, 1]]).to_numpy()
raw_tokens

array([['#', 'DELM'],
       ['اولين', 'ADJ_SUP'],
       ['سياره', 'N_SING'],
       ...,
       ['تماستان', 'N_SING'],
       ['ممنونم', 'V_PRE'],
       ['.', 'DELM']], dtype=object)

In [ ]:
df = []
row_tokens = []
row_tags = []
tags = []
for [token, pos_tag] in raw_tokens:
    tag = pos_tag.split("_")[0]
    if tag not in tags:
        tags.append(tag)
    if token == "#":
        continue
    elif token == ".":
        row_tokens.append(token)
        row_tags.append(tags.index(tag))
        df.append([row_tokens, row_tags])
        row_tokens = []
        row_tags = []
    else:
        row_tokens.append(token)
        row_tags.append(tags.index(tag))
final_df = pd.DataFrame(df)
tags

['DELM',
 'ADJ',
 'N',
 'P',
 'V',
 'CON',
 'ADV',
 'DET',
 'PRO',
 'IF',
 'DEFAULT',
 'QUA',
 'SPEC',
 'INT',
 'PS',
 'PP',
 'MORP',
 'MQUA',
 'OH',
 'AR',
 'NN',
 'NP',
 'OHH',
 'MS']

In [ ]:
final_df.columns = ["tokens", "pos_tags"]
final_df

,tokens,pos_tags
0,"[اولين, سياره, خارج, از, منظومه, شمسي, ديده, ش...","[1, 2, 1, 3, 2, 1, 1, 4, 0]"
1,"[واشنگتن, ـ, خبرگزاري, جمهوري, اسلامي, ستاره ش...","[2, 0, 2, 2, 1, 2, 4, 5, 1, 4, 1, 2, 1, 3, 2, ..."
2,"[طي, سالهاي, اخير, ،, بيش, از, دو, دوجين, سيار...","[3, 2, 1, 0, 1, 3, 2, 2, 2, 3, 2, 2, 2, 2, 4, 0]"
3,"[ليكن, ستاره شناسان, هرگز, اين, سياره ها, را, ...","[5, 2, 6, 7, 2, 3, 6, 1, 4, 5, 6, 3, 2, 2, 0, ..."
4,"[چنين, تصور, مي شد, كه, اين, لرزش, به, علت, جا...","[6, 2, 4, 5, 7, 2, 3, 2, 2, 2, 3, 2, 1, 4, 0]"
...,...,...
83998,"[روند, فعلي, آموزش, بازيگران, جوان, را, مي پسن...","[2, 1, 2, 2, 1, 3, 4, 0, 0, 8, 8, 5, 9, 2, 4, ..."
83999,"[اگر, آموزش, اصولي, و, دقيق, باشد, و, چهره هاي...","[9, 2, 1, 5, 1, 4, 5, 2, 1, 5, 1, 1, 4, 0, 5, ..."
84000,"[بايد, سره, از, ناسره, تفكيك, شود, .]","[4, 2, 3, 2, 2, 4, 0]"
84001,"[از, توضيحاتتان, متشكريم, .]","[3, 2, 4, 0]"


In [ ]:
from datasets import Dataset, DatasetDict, Features, Sequence, ClassLabel, Value
ds_all = Dataset.from_pandas(final_df, features=Features({
                "tokens": Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
                "pos_tags": Sequence(feature=ClassLabel(num_classes=len(tags), names=tags, names_file=None, id=None), length=-1, id=None)
            })).train_test_split(test_size=0.2)
train_test = ds_all["test"].train_test_split(test_size=0.3)
test_validation = train_test["test"].train_test_split(test_size=0.5)
dataset = DatasetDict({
    "train": train_test["train"],
    "test": test_validation["train"],
    "validation": test_validation["test"]})
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'pos_tags'],
        num_rows: 11760
    })
    test: Dataset({
        features: ['tokens', 'pos_tags'],
        num_rows: 2520
    })
    validation: Dataset({
        features: ['tokens', 'pos_tags'],
        num_rows: 2521
    })
})

In [ ]:
dataset["train"].features

{'pos_tags': Sequence(feature=ClassLabel(num_classes=24, names=['DELM', 'ADJ', 'N', 'P', 'V', 'CON', 'ADV', 'DET', 'PRO', 'IF', 'DEFAULT', 'QUA', 'SPEC', 'INT', 'PS', 'PP', 'MORP', 'MQUA', 'OH', 'AR', 'NN', 'NP', 'OHH', 'MS'], names_file=None, id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tags))

In [ ]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
!pip install seqeval
metric = load_metric("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,0.441500,0.212762,0.840378,0.848453,0.844396,0.932243,30.353100,83.056000
2,0.211300,0.181139,0.861299,0.862176,0.861737,0.941198,30.678200,82.176000
3,0.139200,0.179581,0.864921,0.869324,0.867117,0.943691,30.537000,82.556000


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DELM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: N seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarnin

TrainOutput(global_step=2205, training_loss=0.2308026579772534, metrics={'train_runtime': 963.7205, 'train_samples_per_second': 2.288, 'total_flos': 3039695008213248.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 341088, 'init_mem_gpu_alloc_delta': 472429056, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 954439, 'train_mem_gpu_alloc_delta': 1441144832, 'train_mem_cpu_peaked_delta': 158204504, 'train_mem_gpu_peaked_delta': 12033124352})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DELM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: N seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarnin

{'epoch': 3.0,
 'eval_accuracy': 0.943691115238168,
 'eval_f1': 0.8671165929718407,
 'eval_loss': 0.17958082258701324,
 'eval_mem_cpu_alloc_delta': 113595758,
 'eval_mem_cpu_peaked_delta': 44037935,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 546812928,
 'eval_precision': 0.8649207588579146,
 'eval_recall': 0.8693236048953273,
 'eval_runtime': 30.782,
 'eval_samples_per_second': 81.898}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [tags[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [tags[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DELM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: N seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarnin

{'DJ': {'f1': 0.8037866834963711,
  'number': 7909,
  'precision': 0.8024193548387096,
  'recall': 0.8051586799848274},
 'DV': {'f1': 0.66512274201019,
  'number': 1090,
  'precision': 0.6716557530402245,
  'recall': 0.6587155963302752},
 'EFAULT': {'f1': 0.0, 'number': 7, 'precision': 0.0, 'recall': 0.0},
 'ELM': {'f1': 0.9831859004659329,
  'number': 7409,
  'precision': 0.9837837837837838,
  'recall': 0.9825887434201647},
 'ET': {'f1': 0.9502801120448179,
  'number': 1404,
  'precision': 0.9345730027548209,
  'recall': 0.9665242165242165},
 'F': {'f1': 0.9700000000000001,
  'number': 99,
  'precision': 0.9603960396039604,
  'recall': 0.9797979797979798},
 'NT': {'f1': 0.0, 'number': 4, 'precision': 0.0, 'recall': 0.0},
 'ON': {'f1': 0.9329733163913596,
  'number': 6241,
  'precision': 0.9248937175247992,
  'recall': 0.9411953212626182},
 'ORP': {'f1': 0.5, 'number': 92, 'precision': 0.5, 'recall': 0.5},
 'P': {'f1': 0.3684210526315789,
  'number': 28,
  'precision': 0.7,
  'recall':